# Dictionaries and Sets

*📚 Computer Programming II · 👨‍🏫 Dr. Arif Solmaz*


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⏱️ OTURUM ZAMANLAYICI — Bu hücreyi ilk çalıştırın!
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import time as _time, datetime as _dt, json as _json, os as _os
from IPython.display import display, HTML as _HTML

# ── Persistent tracker file (survives kernel restarts in same runtime) ──
_TRACKER_FILE = _os.path.join(
    '/content' if _os.path.isdir('/content') else '/tmp',
    '.cp2_session_Week_06.json'
)

def _load_tracker():
    """Load previous sessions from file."""
    try:
        with open(_TRACKER_FILE, 'r') as f:
            return _json.load(f)
    except (FileNotFoundError, _json.JSONDecodeError, ValueError):
        return {'sessions': [], 'total_heartbeats': 0}

def _save_tracker(data):
    """Persist tracker data to file."""
    try:
        with open(_TRACKER_FILE, 'w') as f:
            _json.dump(data, f)
    except OSError:
        pass

# ── Load any previous session data ──
_tracker = _load_tracker()
_prev_sessions = len(_tracker['sessions'])
_prev_hb = _tracker['total_heartbeats']

# ── Start new session ──
_SESSION_START = _time.time()
_SESSION_START_STR = _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
_HEARTBEATS = [_time.time()]
_CELLS_RUN = [0]
IDLE_THRESHOLD = 300   # 5 min

# Record this session start
_tracker['sessions'].append({
    'start': _SESSION_START_STR,
    'start_ts': _SESSION_START,
    'heartbeats': [_SESSION_START]
})
_save_tracker(_tracker)

def _heartbeat_hook(*args, **kwargs):
    """Record each cell execution — in-memory + file."""
    now = _time.time()
    _HEARTBEATS.append(now)
    _CELLS_RUN[0] += 1
    # Persist every 5 cells to avoid excessive disk IO
    if _CELLS_RUN[0] % 5 == 0:
        try:
            _t = _load_tracker()
            if _t['sessions']:
                _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
                _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
                _t['total_heartbeats'] = _prev_hb + len(_HEARTBEATS)
                _save_tracker(_t)
        except Exception:
            pass

def _calc_active_time(heartbeats=None):
    """Calculate active time from heartbeat list, capping idle gaps."""
    hb = heartbeats or _HEARTBEATS
    if len(hb) < 2: return 0
    active = 0
    for i in range(1, len(hb)):
        gap = hb[i] - hb[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _calc_total_active_time():
    """Calculate total across ALL sessions (multi-day support)."""
    total = 0
    try:
        _t = _load_tracker()
        for sess in _t['sessions'][:-1]:  # all previous sessions
            hb = sess.get('heartbeats', [])
            if len(hb) >= 2:
                for i in range(1, len(hb)):
                    gap = hb[i] - hb[i-1]
                    total += gap if gap <= IDLE_THRESHOLD else 30
    except Exception:
        pass
    # Add current session
    total += _calc_active_time()
    return int(total)

try:
    _ip = get_ipython()
    _ip.events.register('pre_run_cell', _heartbeat_hook)
except Exception: pass

# ── Display ──
_multi = ''
if _prev_sessions > 0:
    _multi = f'<br><span style="font-size:12px;opacity:.75">📂 {_prev_sessions} önceki oturum bulundu — çoklu oturum takibi aktif.</span>'
display(_HTML(f"""<div style='background:linear-gradient(135deg,#667eea,#764ba2);padding:14px 20px;border-radius:10px;color:white;font-family:system-ui;margin:4px 0'>
<b>⏱️ Oturum Başladı</b> — {_SESSION_START_STR}<br>
<span style='font-size:13px;opacity:.85'>Hücre aktiviteniz takip ediliyor. Bitince en alttaki Submit hücresini çalıştırın.</span>{_multi}</div>"""))
print(f'✅ Zamanlayıcı aktif. Idle eşiği: {IDLE_THRESHOLD//60} dk')
if _prev_sessions > 0:
    print(f'📂 Önceki oturumlar: {_prev_sessions} | Toplam heartbeat: {_prev_hb}')

## 🎯 Learning Objectives

- Understand dictionaries as key-value pair collections
- Create dictionaries using various methods
- Access, add, update, and delete dictionary items
- Use dictionary methods: get(), keys(), values(), items()
- Iterate through dictionaries effectively
- Create dictionaries using comprehensions
- Work with nested dictionaries
- Understand sets as unique element collections
- Perform set operations (union, intersection, difference)
- Use set methods and frozensets
- Choose the right data structure for different problems
- Apply dictionaries and sets in engineering contexts

---
## Part 1: Introduction to Dictionaries


A **dictionary** is a built-in Python data structure that stores data as **key-value pairs**. Unlike lists which use integer indices, dictionaries use keys (which can be any immutable type) to access values. Dictionaries are one of Python's most powerful and frequently used data structures.

| Feature | Description |
| --- | --- |
| **Key-Value Pairs** | Each item has a unique key mapped to a value |
| **Mutable** | Can be modified after creation (add, update, delete) |
| **Unique Keys** | Keys must be unique; duplicate keys overwrite |
| **Fast Lookup** | O(1) average time complexity for access |
| **Ordered** | Maintains insertion order (Python 3.7+) |
| **Keys Must Be Immutable** | Keys can be strings, numbers, tuples (not lists) |

### Creating Dictionaries

**Figure 1.1: Creating Empty Dictionaries**

In [ ]:
# Two ways to create empty dictionaries
empty_dict1 = {}
empty_dict2 = dict()

print(f"Empty dict using {{}}: {empty_dict1}")
print(f"Empty dict using dict(): {empty_dict2}")
print(f"Type: {type(empty_dict1)}")
print(f"Length of empty dict: {len(empty_dict1)}")

**Figure 1.2: Creating Dictionaries with Values**

In [ ]:
# Method 1: Literal notation (most common)
student = {
    "name": "Ali Yilmaz",
    "age": 21,
    "department": "Mechatronics",
    "gpa": 3.65
}
print("Student:", student)

# Method 2: Using dict() with keyword arguments
config = dict(host="localhost", port=8080, debug=True)
print("Config:", config)

# Method 3: From list of tuples
pairs = [("a", 1), ("b", 2), ("c", 3)]
from_tuples = dict(pairs)
print("From tuples:", from_tuples)

# Method 4: Using dict.fromkeys()
keys = ["x", "y", "z"]
zeros = dict.fromkeys(keys, 0)
print("From keys:", zeros)

### Valid Key Types

**Figure 1.3: Dictionary Key Types**

In [ ]:
# Keys must be immutable (hashable)
mixed_keys = {
    "string_key": "Value 1",     # String key
    42: "Value 2",                # Integer key
    3.14: "Value 3",              # Float key
    (1, 2): "Value 4",            # Tuple key (immutable)
    True: "Value 5"               # Boolean key
}

print("Dictionary with mixed key types:")
for key, value in mixed_keys.items():
    print(f"  {key!r} ({type(key).__name__}): {value}")

# Note: Lists CANNOT be keys (they are mutable)
# This would cause TypeError: unhashable type: 'list'
# invalid = {[1, 2]: "error"}

### Engineering Example: Sensor Configuration

**Figure 1.4: Sensor Configuration**

In [ ]:
# Real-world engineering: Sensor configuration
temperature_sensor = {
    "sensor_id": "TEMP_001",
    "type": "thermocouple",
    "model": "K-type",
    "unit": "celsius",
    "min_range": -200,
    "max_range": 1250,
    "accuracy": 0.5,
    "location": "Engine Bay",
    "calibration_date": "2024-01-15",
    "is_active": True
}

print("🌡️ Temperature Sensor Configuration")
print("=" * 40)
for key, value in temperature_sensor.items():
    print(f"  {key.replace('_', ' ').title():20}: {value}")

---
## Part 2: Accessing Dictionary Elements


There are multiple ways to access values in a dictionary. Understanding the differences between these methods is crucial for writing robust code.

### Square Bracket Notation

**Figure 2.1: Basic Access**

In [ ]:
# Basic access using square brackets
student = {
    "name": "Ali Yilmaz",
    "age": 21,
    "department": "Mechatronics",
    "courses": ["Programming", "Circuits", "Mechanics"]
}

print(f"Name: {student['name']}")
print(f"Age: {student['age']}")
print(f"Department: {student['department']}")
print(f"Courses: {student['courses']}")
print(f"First course: {student['courses'][0]}")  # Access list within dict

### The get() Method - Safer Access

**Figure 2.2: Using get() Method**

In [ ]:
# get() returns None (or default) if key doesn't exist
student = {"name": "Ali", "age": 21}

# Safe access with get()
print(f"Name: {student.get('name')}")
print(f"GPA: {student.get('gpa')}")  # Returns None
print(f"GPA with default: {student.get('gpa', 0.0)}")
print(f"City with default: {student.get('city', 'Unknown')}")

# Comparison: [] vs get()
print("\n--- Comparison ---")
print("get() with missing key: Returns None or default")
# print(student['gpa'])  # This would raise KeyError!

### Checking Key Existence

**Figure 2.3: Checking Keys**

In [ ]:
# Check if key exists
inventory = {
    "resistor_1k": 500,
    "capacitor_10uF": 200,
    "led_red": 150,
    "arduino_nano": 25
}

# Using 'in' keyword
print(f"'resistor_1k' exists: {'resistor_1k' in inventory}")
print(f"'transistor' exists: {'transistor' in inventory}")
print(f"'motor' not in inventory: {'motor' not in inventory}")

# Conditional access pattern
component = "led_blue"
if component in inventory:
    print(f"{component}: {inventory[component]} units")
else:
    print(f"{component} not in inventory")

> 💡 **Note:** **Best Practice:** Use `get()` when you're not sure if a key exists, and square brackets `[]` when you're certain the key exists and want an error if it doesn't.

---
## Part 3: Modifying Dictionaries


### Adding and Updating Items

**Figure 3.1: Adding and Updating**

In [ ]:
# Adding new key-value pairs
student = {"name": "Ali", "age": 21}
print("Original:", student)

# Add new keys
student["department"] = "Mechatronics"
student["gpa"] = 3.65
print("After adding:", student)

# Update existing value
student["age"] = 22
student["gpa"] = 3.70
print("After updating:", student)

# update() method - add/update multiple items
student.update({"city": "Istanbul", "year": 3, "age": 23})
print("After update():", student)

### Removing Items

**Figure 3.2: Removing Items**

In [ ]:
# Various ways to remove items
data = {"a": 1, "b": 2, "c": 3, "d": 4, "e": 5}
print("Original:", data)

# del - remove by key
del data["a"]
print("After del ['a']:", data)

# pop() - remove and return value
value = data.pop("b")
print(f"Popped 'b': {value}, dict: {data}")

# pop() with default (no error if missing)
value = data.pop("z", "not found")
print(f"Popped 'z': {value}")

# popitem() - remove and return last item
item = data.popitem()
print(f"Popped item: {item}, dict: {data}")

# clear() - remove all items
data.clear()
print("After clear():", data)

### setdefault() Method

**Figure 3.3: Using setdefault()**

In [ ]:
# setdefault() - get value or set default if missing
scores = {"Alice": 85, "Bob": 92}
print("Original:", scores)

# Key exists - returns existing value
alice_score = scores.setdefault("Alice", 0)
print(f"Alice's score: {alice_score}")

# Key doesn't exist - adds and returns default
charlie_score = scores.setdefault("Charlie", 0)
print(f"Charlie's score: {charlie_score}")
print("After setdefault:", scores)

# Useful for counting
word_count = {}
words = ["apple", "banana", "apple", "cherry", "banana", "apple"]
for word in words:
    word_count.setdefault(word, 0)
    word_count[word] += 1
print("\nWord count:", word_count)

---
## Part 4: Dictionary Methods


| Method | Description | Returns |
| --- | --- | --- |
| `keys()` | Get all keys | dict_keys view |
| `values()` | Get all values | dict_values view |
| `items()` | Get all key-value pairs | dict_items view |
| `get(key, default)` | Get value safely | Value or default |
| `pop(key, default)` | Remove and return | Value or default |
| `update(dict)` | Merge dictionaries | None |
| `copy()` | Shallow copy | New dict |

### keys(), values(), items()

**Figure 4.1: Dictionary Views**

In [ ]:
student = {
    "name": "Ali",
    "age": 21,
    "department": "Mechatronics",
    "gpa": 3.65
}

# Get all keys
print("Keys:", list(student.keys()))

# Get all values
print("Values:", list(student.values()))

# Get all items (key-value tuples)
print("Items:", list(student.items()))

# Views are dynamic - they update when dict changes
keys = student.keys()
print(f"\nKeys view: {keys}")
student["city"] = "Istanbul"
print(f"After adding 'city': {keys}")

### Copying Dictionaries

**Figure 4.2: Copying Dictionaries**

In [ ]:
# Be careful with assignment!
original = {"a": 1, "b": 2}
reference = original  # NOT a copy!

reference["c"] = 3
print("After changing reference:")
print(f"  original: {original}")
print(f"  reference: {reference}")

# Proper copy methods
original = {"a": 1, "b": 2}
copy1 = original.copy()       # Shallow copy
copy2 = dict(original)        # Constructor copy

copy1["c"] = 3
print("\nAfter changing copy1:")
print(f"  original: {original}")
print(f"  copy1: {copy1}")

---
## Part 5: Iterating Through Dictionaries


### Different Ways to Iterate

**Figure 5.1: Iterating Dictionaries**

In [ ]:
grades = {"Alice": 85, "Bob": 92, "Charlie": 78, "Diana": 95}

# Method 1: Iterate over keys (default)
print("Method 1 - Keys only:")
for name in grades:
    print(f"  {name}")

# Method 2: Iterate over keys explicitly
print("\nMethod 2 - Using keys():")
for name in grades.keys():
    print(f"  {name}: {grades[name]}")

# Method 3: Iterate over values
print("\nMethod 3 - Values only:")
for score in grades.values():
    print(f"  Score: {score}")

# Method 4: Iterate over items (most common)
print("\nMethod 4 - Using items():")
for name, score in grades.items():
    print(f"  {name}: {score}")

### Practical Iteration Examples

**Figure 5.2: Practical Examples**

In [ ]:
scores = {"Alice": 85, "Bob": 92, "Charlie": 78, "Diana": 95, "Eve": 88}

# Calculate statistics
total = sum(scores.values())
average = total / len(scores)
print(f"Average score: {average:.1f}")

# Find highest scorer
top_student = max(scores, key=scores.get)
print(f"Top student: {top_student} ({scores[top_student]})")

# Filter passing students (>= 80)
passing = {name: score for name, score in scores.items() if score >= 80}
print(f"Passing students: {passing}")

# Sort by score
sorted_by_score = sorted(scores.items(), key=lambda x: x[1], reverse=True)
print("\nRanking:")
for rank, (name, score) in enumerate(sorted_by_score, 1):
    print(f"  {rank}. {name}: {score}")

> 💡 **Note:** **Don't modify dict size while iterating!** Adding or removing keys during iteration causes `RuntimeError`. Create a copy or list of keys first.

---
## Part 6: Dictionary Comprehensions


Dictionary comprehensions provide a concise way to create dictionaries, similar to list comprehensions.

### Basic Syntax

```
{key_expr: value_expr for item in iterable}

# With condition:
{key_expr: value_expr for item in iterable if condition}
```

**Figure 6.1: Basic Comprehensions**

In [ ]:
# Create dict from range
squares = {x: x**2 for x in range(1, 6)}
print("Squares:", squares)

# Create from two lists
names = ["Alice", "Bob", "Charlie"]
ages = [25, 30, 35]
name_age = {name: age for name, age in zip(names, ages)}
print("Name-Age:", name_age)

# Character frequency
text = "hello"
freq = {char: text.count(char) for char in set(text)}
print("Frequency:", freq)

# ASCII values
letters = "ABCDE"
ascii_vals = {ch: ord(ch) for ch in letters}
print("ASCII:", ascii_vals)

### Filtering with Comprehensions

**Figure 6.2: Filtering**

In [ ]:
# Filter even squares
even_squares = {x: x**2 for x in range(1, 11) if x % 2 == 0}
print("Even squares:", even_squares)

# Filter from existing dict
grades = {"Alice": 85, "Bob": 72, "Charlie": 91, "Diana": 68, "Eve": 88}

# Only passing grades (>= 75)
passing = {name: score for name, score in grades.items() if score >= 75}
print("Passing:", passing)

# Convert to letter grades
def to_letter(score):
    if score >= 90: return 'A'
    elif score >= 80: return 'B'
    elif score >= 70: return 'C'
    elif score >= 60: return 'D'
    return 'F'

letter_grades = {name: to_letter(score) for name, score in grades.items()}
print("Letter grades:", letter_grades)

### Transforming Dictionaries

**Figure 6.3: Transformations**

In [ ]:
# Swap keys and values
original = {"a": 1, "b": 2, "c": 3}
swapped = {v: k for k, v in original.items()}
print("Swapped:", swapped)

# Convert keys to uppercase
data = {"name": "Ali", "city": "Istanbul", "country": "Turkey"}
upper_keys = {k.upper(): v for k, v in data.items()}
print("Upper keys:", upper_keys)

# Prices with tax
prices = {"apple": 1.20, "banana": 0.50, "orange": 0.80}
tax_rate = 0.18
with_tax = {item: round(price * (1 + tax_rate), 2) 
            for item, price in prices.items()}
print("With 18% tax:", with_tax)

---
## Part 7: Nested Dictionaries


Dictionaries can contain other dictionaries as values, creating hierarchical data structures perfect for complex real-world data.

**Figure 7.1: Creating Nested Dicts**

In [ ]:
# Nested dictionary - student records
students = {
    "S001": {
        "name": "Ali Yilmaz",
        "age": 21,
        "courses": {
            "Programming": 85,
            "Circuits": 78,
            "Mechanics": 92
        }
    },
    "S002": {
        "name": "Ayse Kaya",
        "age": 20,
        "courses": {
            "Programming": 91,
            "Circuits": 88,
            "Mechanics": 85
        }
    }
}

# Access nested values
print(f"Student S001 name: {students['S001']['name']}")
print(f"S001 Programming grade: {students['S001']['courses']['Programming']}")

### Iterating Nested Dictionaries

**Figure 7.2: Iterating Nested Dicts**

In [ ]:
students = {
    "S001": {"name": "Ali", "courses": {"Math": 85, "Physics": 90}},
    "S002": {"name": "Ayse", "courses": {"Math": 92, "Physics": 88}},
    "S003": {"name": "Mehmet", "courses": {"Math": 78, "Physics": 82}}
}

# Print all student information
print("📚 Student Records")
print("=" * 40)
for student_id, info in students.items():
    print(f"\n{student_id}: {info['name']}")
    for course, grade in info['courses'].items():
        print(f"  • {course}: {grade}")

# Calculate averages
print("\n📊 Student Averages:")
for student_id, info in students.items():
    avg = sum(info['courses'].values()) / len(info['courses'])
    print(f"  {info['name']}: {avg:.1f}")

### Modifying Nested Dictionaries

**Figure 7.3: Modifying Nested Dicts**

In [ ]:
students = {
    "S001": {"name": "Ali", "courses": {"Math": 85}}
}

# Add new course
students["S001"]["courses"]["Physics"] = 90
print("After adding Physics:", students)

# Add new student
students["S002"] = {
    "name": "Ayse",
    "courses": {"Math": 92, "Physics": 88}
}
print("\nAfter adding S002:", students)

# Update nested value
students["S001"]["courses"]["Math"] = 88
print("\nAfter updating Math:", students["S001"])

# Safe nested access with get()
grade = students.get("S003", {}).get("courses", {}).get("Math", "N/A")
print(f"\nS003 Math grade: {grade}")

---
## Part 8: Introduction to Sets


A **set** is an unordered collection of **unique** elements. Sets are useful when you need to eliminate duplicates, test membership, or perform mathematical set operations.

| Feature | Description |
| --- | --- |
| **Unique Elements** | No duplicate values allowed |
| **Unordered** | No index access, order not guaranteed |
| **Mutable** | Can add/remove elements |
| **Hashable Elements** | Elements must be immutable |
| **Fast Operations** | O(1) membership testing |

### Creating Sets

**Figure 8.1: Creating Sets**

In [ ]:
# Create with curly braces
fruits = {"apple", "banana", "cherry"}
print("Fruits:", fruits)

# Create with set() constructor
numbers = set([1, 2, 3, 4, 5])
print("Numbers:", numbers)

# From string (unique characters)
chars = set("hello")
print("Unique chars in 'hello':", chars)

# Empty set (NOT {} - that's empty dict!)
empty_set = set()
print(f"Empty set: {empty_set}, type: {type(empty_set)}")

# Duplicates are automatically removed
with_duplicates = {1, 2, 2, 3, 3, 3, 4}
print("After removing duplicates:", with_duplicates)

### Adding and Removing Elements

**Figure 8.2: Modifying Sets**

In [ ]:
colors = {"red", "green", "blue"}
print("Original:", colors)

# add() - add single element
colors.add("yellow")
print("After add('yellow'):", colors)

# Adding duplicate has no effect
colors.add("red")
print("After add('red') again:", colors)

# update() - add multiple elements
colors.update(["purple", "orange"])
print("After update():", colors)

# remove() - raises error if not found
colors.remove("yellow")
print("After remove('yellow'):", colors)

# discard() - no error if not found
colors.discard("pink")  # No error
print("After discard('pink'):", colors)

# pop() - remove arbitrary element
popped = colors.pop()
print(f"Popped: {popped}, remaining: {colors}")

### Membership Testing

**Figure 8.3: Fast Membership Testing**

In [ ]:
# Sets are much faster than lists for membership testing
valid_users = {"alice", "bob", "charlie", "diana", "eve"}

# Check membership
print(f"'alice' in set: {'alice' in valid_users}")
print(f"'frank' in set: {'frank' in valid_users}")

# Practical example: validate input
def validate_user(username):
    return username.lower() in valid_users

test_users = ["Alice", "Frank", "Bob", "Unknown"]
for user in test_users:
    status = "✓ Valid" if validate_user(user) else "✗ Invalid"
    print(f"  {user}: {status}")

---
## Part 9: Set Operations


Sets support mathematical operations like union, intersection, and difference.

**Figure 9.1: Set Operations**

In [ ]:
A = {1, 2, 3, 4, 5}
B = {4, 5, 6, 7, 8}

print(f"Set A: {A}")
print(f"Set B: {B}")

# Union - all elements from both sets
print(f"\nUnion (A | B): {A | B}")
print(f"Union (A.union(B)): {A.union(B)}")

# Intersection - common elements
print(f"\nIntersection (A & B): {A & B}")
print(f"Intersection (A.intersection(B)): {A.intersection(B)}")

# Difference - elements in A but not in B
print(f"\nDifference (A - B): {A - B}")
print(f"Difference (B - A): {B - A}")

# Symmetric Difference - elements in either but not both
print(f"\nSymmetric Difference (A ^ B): {A ^ B}")

### Subset and Superset

**Figure 9.2: Subset/Superset**

In [ ]:
A = {1, 2, 3}
B = {1, 2, 3, 4, 5}
C = {1, 2, 3}

# Subset - all elements of A in B?
print(f"A = {A}")
print(f"B = {B}")
print(f"A is subset of B: {A.issubset(B)}")
print(f"A <= B: {A <= B}")
print(f"A < B (proper subset): {A < B}")

# Superset - B contains all of A?
print(f"\nB is superset of A: {B.issuperset(A)}")
print(f"B >= A: {B >= A}")

# Equal sets
print(f"\nA == C: {A == C}")

# Disjoint - no common elements?
X = {1, 2, 3}
Y = {4, 5, 6}
print(f"\nX = {X}, Y = {Y}")
print(f"X and Y are disjoint: {X.isdisjoint(Y)}")

### Practical Example: Finding Common Skills

**Figure 9.3: Practical Set Operations**

In [ ]:
# Job candidates and their skills
alice_skills = {"Python", "JavaScript", "SQL", "Git"}
bob_skills = {"Python", "Java", "SQL", "Docker"}
required_skills = {"Python", "SQL", "Git"}

print("Alice's skills:", alice_skills)
print("Bob's skills:", bob_skills)
print("Required skills:", required_skills)

# Who has all required skills?
print(f"\nAlice has all required: {required_skills.issubset(alice_skills)}")
print(f"Bob has all required: {required_skills.issubset(bob_skills)}")

# What skills do both have?
common = alice_skills & bob_skills
print(f"\nCommon skills: {common}")

# What unique skills does each have?
alice_unique = alice_skills - bob_skills
bob_unique = bob_skills - alice_skills
print(f"Alice's unique: {alice_unique}")
print(f"Bob's unique: {bob_unique}")

# All skills combined
all_skills = alice_skills | bob_skills
print(f"All skills: {all_skills}")

---
## Part 10: Set Methods


| Method | Description | Returns |
| --- | --- | --- |
| `add(elem)` | Add element | None |
| `remove(elem)` | Remove element (error if missing) | None |
| `discard(elem)` | Remove element (no error) | None |
| `pop()` | Remove arbitrary element | Element |
| `clear()` | Remove all elements | None |
| `copy()` | Shallow copy | New set |
| `union()` | Return union | New set |
| `intersection()` | Return intersection | New set |
| `difference()` | Return difference | New set |

### In-place Update Methods

**Figure 10.1: In-place Updates**

In [ ]:
A = {1, 2, 3, 4}
B = {3, 4, 5, 6}

# update() - in-place union
A_copy = A.copy()
A_copy.update(B)  # Same as A_copy |= B
print(f"update (union): {A_copy}")

# intersection_update() - in-place intersection
A_copy = A.copy()
A_copy.intersection_update(B)  # Same as A_copy &= B
print(f"intersection_update: {A_copy}")

# difference_update() - in-place difference
A_copy = A.copy()
A_copy.difference_update(B)  # Same as A_copy -= B
print(f"difference_update: {A_copy}")

# symmetric_difference_update()
A_copy = A.copy()
A_copy.symmetric_difference_update(B)  # Same as A_copy ^= B
print(f"symmetric_difference_update: {A_copy}")

### Set Comprehensions

**Figure 10.2: Set Comprehensions**

In [ ]:
# Set comprehension syntax
squares = {x**2 for x in range(1, 6)}
print("Squares:", squares)

# With condition
even_squares = {x**2 for x in range(1, 11) if x % 2 == 0}
print("Even squares:", even_squares)

# Unique words from sentence
sentence = "the quick brown fox jumps over the lazy dog"
unique_words = {word for word in sentence.split()}
print("Unique words:", unique_words)

# Unique first letters
first_letters = {word[0].upper() for word in sentence.split()}
print("First letters:", first_letters)

---
## Part 11: Frozen Sets


A **frozenset** is an immutable version of a set. Once created, it cannot be modified. Frozensets can be used as dictionary keys or elements of other sets.

**Figure 11.1: Frozen Sets**

In [ ]:
# Create a frozenset
frozen = frozenset([1, 2, 3, 4, 5])
print(f"Frozenset: {frozen}")
print(f"Type: {type(frozen)}")

# Frozensets support same operations (non-modifying)
other = frozenset([4, 5, 6, 7])
print(f"\nUnion: {frozen | other}")
print(f"Intersection: {frozen & other}")
print(f"Difference: {frozen - other}")

# But they cannot be modified
# frozen.add(6)  # AttributeError!
# frozen.remove(1)  # AttributeError!

# Use as dictionary key
permissions = {
    frozenset(["read"]): "viewer",
    frozenset(["read", "write"]): "editor",
    frozenset(["read", "write", "admin"]): "admin"
}
user_perms = frozenset(["read", "write"])
print(f"\nUser role: {permissions[user_perms]}")

---
## Part 12: Engineering Applications


#### Sensor Data Management

**Figure 12.1: Sensor Management**

In [ ]:
# Sensor data management system
sensors = {
    "TEMP_01": {"type": "temperature", "unit": "°C", "readings": [22.5, 23.1, 22.8]},
    "PRESS_01": {"type": "pressure", "unit": "kPa", "readings": [101.3, 101.5, 101.2]},
    "HUMID_01": {"type": "humidity", "unit": "%", "readings": [45, 47, 46]}
}

def analyze_sensor(sensor_id):
    if sensor_id not in sensors:
        return None
    data = sensors[sensor_id]
    readings = data["readings"]
    return {
        "sensor_id": sensor_id,
        "type": data["type"],
        "min": min(readings),
        "max": max(readings),
        "avg": sum(readings) / len(readings),
        "unit": data["unit"]
    }

# Analyze all sensors
print("📊 Sensor Analysis Report")
print("=" * 50)
for sensor_id in sensors:
    stats = analyze_sensor(sensor_id)
    print(f"\n{stats['sensor_id']} ({stats['type']}):")
    print(f"  Range: {stats['min']} - {stats['max']} {stats['unit']}")
    print(f"  Average: {stats['avg']:.2f} {stats['unit']}")

#### Component Inventory System

**Figure 12.2: Inventory System**

In [ ]:
# Component inventory with sets for categories
inventory = {
    "resistors": {"1k", "10k", "100k", "1M"},
    "capacitors": {"10uF", "100uF", "1000uF"},
    "microcontrollers": {"Arduino Nano", "ESP32", "STM32"}
}

# Project requirements
project_a_needs = {"1k", "10k", "10uF", "Arduino Nano"}
project_b_needs = {"10k", "100k", "100uF", "ESP32", "STM32"}

# Check what's available
all_components = set()
for category in inventory.values():
    all_components.update(category)

print("All available components:", all_components)

# Check project requirements
for project, needs in [("Project A", project_a_needs), ("Project B", project_b_needs)]:
    available = needs & all_components
    missing = needs - all_components
    print(f"\n{project}:")
    print(f"  Available: {available}")
    print(f"  Missing: {missing if missing else 'None'}")

#### Data Validation

**Figure 12.3: Data Validation**

In [ ]:
# Validate sensor readings
def validate_reading(sensor_type, value):
    """Validate sensor readings against allowed ranges"""
    valid_ranges = {
        "temperature": (-40, 85),
        "pressure": (80, 120),
        "humidity": (0, 100),
        "voltage": (0, 5)
    }
    
    if sensor_type not in valid_ranges:
        return {"valid": False, "error": "Unknown sensor type"}
    
    min_val, max_val = valid_ranges[sensor_type]
    if min_val <= value <= max_val:
        return {"valid": True, "value": value}
    else:
        return {"valid": False, "error": f"Out of range ({min_val}-{max_val})"}

# Test readings
test_data = [
    ("temperature", 25.5),
    ("temperature", 100),
    ("humidity", 45),
    ("voltage", 3.3),
    ("voltage", 12)
]

print("📋 Validation Results:")
for sensor_type, value in test_data:
    result = validate_reading(sensor_type, value)
    status = "✓" if result["valid"] else "✗"
    msg = value if result["valid"] else result["error"]
    print(f"  {status} {sensor_type}({value}): {msg}")

---
## ❌ Common Mistakes to Avoid


These are the most frequent errors students make with dictionaries and sets. Study them before the exercises!

**Accessing a key that doesn't exist**

`my_dict["missing_key"]` → `KeyError`! Use `my_dict.get("key", default)` to safely access keys, or check with `"key" in my_dict` first.

**Using mutable types as dictionary keys**

`{[1,2]: "value"}` → `TypeError`! Lists can't be keys because they're mutable. Use tuples instead: `{(1,2): "value"}`.

**Modifying a dictionary while iterating**

`for k in d: del d[k]` → `RuntimeError`! Iterate over a copy of the keys instead: `for k in list(d.keys()):`

**Expecting order in sets**

                    Sets are unordered — `{3, 1, 2}` may print as `{1, 2, 3}`. Don't rely on element order. If order matters, use a list or `sorted()`.

**Creating an empty set with `{}`**

`x = {}` creates an empty *dictionary*, not a set! Use `x = set()` for an empty set.

---
# 📝 Exercises


### Exercise 1: Create a Dictionary  (Easy)

Create a dictionary called `person` with keys "name", "age", and "city".

**Expected Output:**
```
{'name': 'Ali', 'age': 25, 'city': 'Istanbul'}
```

<details>
<summary>💡 Hints</summary>

- Use curly braces: `{"key": value}`
- Separate pairs with commas
- Example: `{"name": "Ali", "age": 25, ...}`
</details>

In [ ]:
# ✏️ [EX1]
# Create the dictionary

print(person)

### Exercise 2: Access with get()  (Easy)

Use `get()` to access the "country" key with default "Unknown".

**Expected Output:**
```
Country: Unknown
```

<details>
<summary>💡 Hints</summary>

- Syntax: `dict.get(key, default)`
- Returns default if key not found
- Example: `person.get("country", "Unknown")`
</details>

In [ ]:
# ✏️ [EX2]
person = {"name": "Ali", "age": 25, "city": "Istanbul"}
# Use get() with default value
country = 
print(f"Country: {country}")

### Exercise 3: Add and Update  (Easy)

Add key "email" and update "age" to 26.

**Expected Output:**
```
{'name': 'Ali', 'age': 26, 'email': '[email protected]'}
```

<details>
<summary>💡 Hints</summary>

- Add/update: `dict[key] = value`
- Add email: `person["email"] = "..."`
- Update age: `person["age"] = 26`
</details>

In [ ]:
# ✏️ [EX3]
person = {"name": "Ali", "age": 25}
# Add email and update age


print(person)

### Exercise 4: Iterate Items  (Easy)

Print each key-value pair on a separate line using items().

**Expected Output:**
```
name: Ali
age: 25
city: Istanbul
```

<details>
<summary>💡 Hints</summary>

- Use `for key, value in dict.items():`
- Print: `f"{key}: {value}"`
- items() returns key-value tuples
</details>

In [ ]:
# ✏️ [EX4]
person = {"name": "Ali", "age": 25, "city": "Istanbul"}
# Print each key-value pair

### Exercise 5: Dict Comprehension  (Medium)

Create a dict of squares for numbers 1-5 using comprehension.

**Expected Output:**
```
{1: 1, 2: 4, 3: 9, 4: 16, 5: 25}
```

<details>
<summary>💡 Hints</summary>

- Syntax: `{key: value for item in iterable}`
- Example: `{x: x**2 for x in range(1, 6)}`
- Creates key-value pairs in one line
</details>

In [ ]:
# ✏️ [EX5]
# Create squares dict using comprehension
squares = 
print(squares)

### Exercise 6: Word Counter  (Medium)

Count the frequency of each word in the string.

**Expected Output:**
```
{'hello': 2, 'world': 1}
```

<details>
<summary>💡 Hints</summary>

- Split text: `text.split()`
- Loop through words, use `get(word, 0) + 1`
- Or: `word_count[word] = word_count.get(word, 0) + 1`
</details>

In [ ]:
# ✏️ [EX6]
text = "hello world hello"
# Count word frequency
word_count = {}

print(word_count)

### Exercise 7: Nested Dict Access  (Medium)

Access the math grade from the nested dictionary.

**Expected Output:**
```
Math grade: 85
```

<details>
<summary>💡 Hints</summary>

- Chain brackets: `dict[key1][key2]`
- Access: `student["grades"]["math"]`
- First get inner dict, then the value
</details>

In [ ]:
# ✏️ [EX7]
student = {
    "name": "Ali",
    "grades": {"math": 85, "physics": 90}
}
# Access math grade
math_grade = 
print(f"Math grade: {math_grade}")

### Exercise 8: Create a Set  (Easy)

Create a set from a list that has duplicates.

**Expected Output:**
```
{1, 2, 3, 4, 5}
```

<details>
<summary>💡 Hints</summary>

- Convert list to set: `set(list)`
- Sets automatically remove duplicates
- Example: `set(numbers)`
</details>

In [ ]:
# ✏️ [EX8]
numbers = [1, 2, 2, 3, 3, 3, 4, 5, 5]
# Create a set from the list
unique = 
print(unique)

### Exercise 9: Set Union  (Easy)

Find the union of sets A and B.

**Expected Output:**
```
{1, 2, 3, 4, 5, 6}
```

<details>
<summary>💡 Hints</summary>

- Union method: `A.union(B)`
- Or use operator: `A | B`
- Returns all unique elements from both sets
</details>

In [ ]:
# ✏️ [EX9]
A = {1, 2, 3, 4}
B = {3, 4, 5, 6}
# Find union
result = 
print(result)

### Exercise 10: Set Intersection  (Easy)

Find elements common to both sets.

**Expected Output:**
```
{3, 4}
```

<details>
<summary>💡 Hints</summary>

- Intersection: `A.intersection(B)`
- Or use operator: `A & B`
- Returns elements in both sets
</details>

In [ ]:
# ✏️ [EX10]
A = {1, 2, 3, 4}
B = {3, 4, 5, 6}
# Find intersection
common = 
print(common)

### Exercise 11: Set Difference  (Medium)

Find elements in A but not in B.

**Expected Output:**
```
{1, 2}
```

<details>
<summary>💡 Hints</summary>

- Difference: `A.difference(B)`
- Or use operator: `A - B`
- Returns elements only in A
</details>

In [ ]:
# ✏️ [EX11]
A = {1, 2, 3, 4}
B = {3, 4, 5, 6}
# Find A - B
only_in_A = 
print(only_in_A)

### Exercise 12: Filter Dict  (Medium)

Filter grades to only include scores >= 80.

**Expected Output:**
```
{'Alice': 85, 'Charlie': 92}
```

<details>
<summary>💡 Hints</summary>

- Dict comprehension with condition
- Syntax: `{k: v for k, v in dict.items() if condition}`
- Example: `{k: v for k, v in grades.items() if v >= 80}`
</details>

In [ ]:
# ✏️ [EX12]
grades = {"Alice": 85, "Bob": 72, "Charlie": 92, "Diana": 68}
# Filter to >= 80 using comprehension
passing = 
print(passing)

### Exercise 13: Check Subset  (Medium)

Check if required_skills is a subset of candidate_skills.

**Expected Output:**
```
Has all required skills: True
```

<details>
<summary>💡 Hints</summary>

- Use `set.issubset(other)`
- Or operator: `A <= B`
- Example: `required_skills.issubset(candidate_skills)`
</details>

In [ ]:
# ✏️ [EX13]
required_skills = {"Python", "SQL"}
candidate_skills = {"Python", "SQL", "JavaScript", "Git"}
# Check if candidate has all required skills
has_all = 
print(f"Has all required skills: {has_all}")

### Exercise 14: Merge Dictionaries  (Challenge)

Merge two dictionaries into one.

**Expected Output:**
```
{'a': 1, 'b': 2, 'c': 3, 'd': 4}
```

<details>
<summary>💡 Hints</summary>

- Use `dict1 | dict2` (Python 3.9+) or `{**dict1, **dict2}`
</details>

In [ ]:
# ✏️ [EX14]
dict1 = {"a": 1, "b": 2}
dict2 = {"c": 3, "d": 4}
# Merge the dictionaries
merged = 
print(merged)

### Exercise 15: Invert Dictionary  (Challenge)

Swap keys and values in the dictionary.

**Expected Output:**
```
{1: 'a', 2: 'b', 3: 'c'}
```

<details>
<summary>💡 Hints</summary>

- Use dict comprehension: `{v: k for k, v in original.items()}`
</details>

In [ ]:
# ✏️ [EX15]
original = {"a": 1, "b": 2, "c": 3}
# Swap keys and values
inverted = 
print(inverted)

### Exercise 🌉: Bridge Exercise: Sneak Peek at Week 7  (Preview)

**Next week: String Methods!** You have a dictionary of raw sensor log entries as strings. Extracting meaningful data from them is awkward with indexing. What if strings had built-in tools for splitting, stripping, and searching?

**Expected Output:**
```
Raw log: '  TEMP: 24.5C | STATUS: OK  '
Manual extraction is fragile and error-prone!

Character-by-character parsing? 😫
sensor_type = TEMP: 24 (wrong — includes extra chars)

Next week: .split(), .strip(), .replace() make this easy!
```

<details>
<summary>💡 Hints</summary>

- String indexing with `[2:6]` breaks if the format changes
- What if "TEMP" was "TEMPERATURE"? All your index positions break
- Next week: `log.strip().split("|")` handles any length cleanly
</details>

In [ ]:
# ✏️ [EXBridge]
# Bridge Exercise: Raw Text Parsing Problem
# Sensor log data stored as raw strings in a dictionary

logs = {
    "sensor_01": "  TEMP: 24.5C | STATUS: OK  ",
    "sensor_02": "  HUMIDITY: 65% | STATUS: WARNING  ",
}

raw = logs["sensor_01"]
print(f"Raw log: '{raw}'")
print("Manual extraction is fragile and error-prone!\n")

# Try to extract sensor type using indexing...
sensor_type = raw[2:10]  # Hardcoded positions!
print("Character-by-character parsing? 😫")
print(f"sensor_type = {sensor_type} (wrong — includes extra chars)")

# THINK: What if the label was "TEMPERATURE" instead of "TEMP"?
# All our hardcoded index positions would break!
print("\nNext week: .split(), .strip(), .replace() make this easy!")

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,12}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-12 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
_ORIGINAL_HASHES = {
    "ex1": "ea59a8f7c6fe6fa1e67e2664872ee4cf",
    "ex2": "3f932be8f0f804c76e10e8af7d30431f",
    "ex3": "c85cd6ac1992901583c8a830d8c1768e",
    "ex4": "430744f7bc3b2cc14993c7fe7532160f",
    "ex5": "481343a0f93e38131bb83b044e9d4020",
    "ex6": "124e41748edd4243df2aa9524901594f",
    "ex7": "828ff1a23b28890e0a98a237769bf24b",
    "ex8": "01e869f4a07117484d661e836911c0a3",
    "ex9": "f6bbf66adc27332c048ce2f67b822a13",
    "ex10": "fe2bb5dd7fa2bd6f5aa41aaf69bc519b",
    "ex11": "d47ff7468d1c7b565b117d796bac465d",
    "ex12": "9adb5688dbbba801acbb39da5b8146e5",
    "ex13": "9f8f5fe0682af9d6c0357b8d16aa7973",
    "ex14": "b3052fe381983bffa86c9d9fd4162247",
    "ex15": "ae687b723b899bc60e652380f5b0e707"
}

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request, hashlib
import time as _time, datetime as _dt

WEEK = "Week_06"
URL  = "https://script.google.com/macros/s/AKfycbyf1D3HGSAX4MoIhNlAuWlGrFyyvbM5MIv7ZsLxrVDlATUihrRGEAaibvIZYlCfd8Me/exec"

# ══════════════════════════════════════════════════════════
# SELF-HEALING TIMER — works even if Cell 1 was skipped
# ══════════════════════════════════════════════════════════
_TRACKER_FILE = os.path.join(
    '/content' if os.path.isdir('/content') else '/tmp',
    f'.cp2_session_{WEEK}.json'
)
IDLE_THRESHOLD = 300

def _sh_calc_active(heartbeats):
    """Calculate active seconds from a heartbeat list."""
    if len(heartbeats) < 2: return 0
    active = 0
    for i in range(1, len(heartbeats)):
        gap = heartbeats[i] - heartbeats[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _collect_timing():
    """
    Collect timing data from all available sources:
      1. In-memory variables (timer cell ran in this kernel)
      2. Persistent tracker file (previous sessions / kernel restarts)
      3. Minimal fallback (nothing available)
    """
    _info = {
        'active_time': 0, 'wall_time': 0, 'cells_run': 0,
        'heartbeat_count': 0, 'session_count': 0,
        'session_start': '', 'session_end': _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'none'
    }

    # ── Source 1: In-memory (timer cell ran this kernel) ──
    _has_memory = False
    try:
        _s = _SESSION_START        # noqa: F821
        _hb = list(_HEARTBEATS)    # noqa: F821
        _cr = _CELLS_RUN[0]        # noqa: F821
        _has_memory = True
        _info['active_time'] = _sh_calc_active(_hb)
        _info['wall_time'] = int(_time.time() - _s)
        _info['cells_run'] = _cr
        _info['heartbeat_count'] = len(_hb)
        _info['session_start'] = _SESSION_START_STR  # noqa: F821
        _info['source'] = 'memory'
    except NameError:
        pass

    # ── Source 2: Persistent file (adds previous sessions) ──
    try:
        with open(_TRACKER_FILE, 'r') as f:
            _t = json.load(f)
        sessions = _t.get('sessions', [])
        _info['session_count'] = len(sessions)

        if not _has_memory and sessions:
            # Timer cell was NOT run — reconstruct from file
            last = sessions[-1]
            hb = last.get('heartbeats', [])
            _info['active_time'] = _sh_calc_active(hb)
            _info['wall_time'] = int(hb[-1] - hb[0]) if len(hb) >= 2 else 0
            _info['cells_run'] = last.get('cells_run', len(hb))
            _info['heartbeat_count'] = len(hb)
            _info['session_start'] = last.get('start', '')
            _info['source'] = 'file-current'

        # Add previous sessions' active time
        if len(sessions) > 1:
            prev_active = 0
            prev_cells = 0
            prev_hb = 0
            cutoff = -1 if _has_memory else -1  # exclude current session
            for sess in sessions[:cutoff]:
                hb = sess.get('heartbeats', [])
                prev_active += _sh_calc_active(hb)
                prev_cells += sess.get('cells_run', len(hb))
                prev_hb += len(hb)
            _info['prev_active_time'] = prev_active
            _info['prev_cells_run'] = prev_cells
            _info['prev_heartbeats'] = prev_hb
            _info['total_active_time'] = _info['active_time'] + prev_active
            _info['total_cells_run'] = _info['cells_run'] + prev_cells
            if _info['source'] == 'none':
                _info['source'] = 'file-prev'

    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError):
        pass

    # Ensure totals exist
    _info.setdefault('total_active_time', _info['active_time'])
    _info.setdefault('total_cells_run', _info['cells_run'])
    return _info

# ── Persist final heartbeat snapshot before collecting ──
try:
    _t = json.load(open(_TRACKER_FILE))
    if _t['sessions']:
        _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
        _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
        json.dump(_t, open(_TRACKER_FILE, 'w'))
except Exception:
    pass

_timing = _collect_timing()

# ── Display timing ──
if _timing['source'] == 'none':
    print("⚠️  Zamanlayıcı verisi bulunamadı — ilk hücreyi çalıştırmayı unutmayın.")
    print("    Gönderim yine de yapılacak (süre: 0 olarak kaydedilir).")
else:
    _a = _timing['active_time']
    _w = _timing['wall_time']
    print(f"⏱️  Bu oturum — Aktif: {_a//60}m {_a%60}s  |  Duvar: {_w//60}m {_w%60}s")
    print(f"🔢  Hücreler: {_timing['cells_run']}  |  Heartbeat: {_timing['heartbeat_count']}")
    if _timing['session_count'] > 1:
        _ta = _timing['total_active_time']
        print(f"📂  Toplam {_timing['session_count']} oturum — Toplam aktif: {_ta//60}m {_ta%60}s  |  Toplam hücre: {_timing['total_cells_run']}")
    if _timing['total_active_time'] < 120:
        print("⚠️  Toplam aktif süre < 2 dk — çalıştırdığınız hücre sayısı düşük olabilir.")

_time_on_page = _timing['total_active_time']

# ══════════════════════════════════════════════════════════
# STUDENT INFO CHECK
# ══════════════════════════════════════════════════════════
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın (öğrenci bilgileri).")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın — bazı alanlar boş.")

# ══════════════════════════════════════════════════════════
# EXERCISE ANSWER COLLECTION (improved with retry)
# ══════════════════════════════════════════════════════════
_EX_PATTERN = re.compile(r'#\s*✏️\s*\[EX(\d+)\]')

def _extract_exercise(src):
    """Try to extract exercise ID and code from a source string."""
    m = _EX_PATTERN.match(src)
    if not m: return None, None
    ex_id = 'ex' + m.group(1)
    clean = '\n'.join(src.split('\n')[1:]).strip()
    return ex_id, clean

_answers = {}

# ── Method 1: In[] list (most reliable for Run-All) ──
try:
    for _src in In:
        if not _src: continue
        _eid, _code = _extract_exercise(_src)
        if _eid:
            _answers[_eid] = {
                'code': _code,
                'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
            }
except NameError:
    pass

# ── Method 2: IPython history_manager (flush first!) ──
if not _answers:
    try:
        _ipy = get_ipython()
        # Force flush so Run-All history is fully written
        try: _ipy.history_manager.db.commit()
        except Exception: pass
        _time.sleep(0.5)  # small delay for DB sync
        for _sess, _line, _src in _ipy.history_manager.get_range(output=False):
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                }
    except Exception:
        pass

# ── Method 3: Read notebook file (VS Code / local Jupyter) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir('.') if f.endswith('.ipynb') and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), 'r', encoding='utf-8') as _f:
            _nb = json.load(_f)
        for _cell in _nb['cells']:
            if _cell['cell_type'] != 'code': continue
            _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                }

# ── Method 4: Colab notebook file at /content/ ──
if not _answers:
    try:
        _colab_candidates = [f for f in os.listdir('/content') if f.endswith('.ipynb') and WEEK in f]
        if _colab_candidates:
            _nb_path = os.path.join('/content', _colab_candidates[0])
            with open(_nb_path, 'r', encoding='utf-8') as _f:
                _nb = json.load(_f)
            for _cell in _nb['cells']:
                if _cell['cell_type'] != 'code': continue
                _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
                _eid, _code = _extract_exercise(_src)
                if _eid:
                    _answers[_eid] = {
                        'code': _code,
                        'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                    }
    except Exception:
        pass

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys(), key=lambda x: int(x[2:]) if x[2:].isdigit() else 999))}")

if not _answers:
    print("\n⚠️  Hiçbir exercise yanıtı bulunamadı!")
    print("Submit'ten ÖNCE tüm exercise hücrelerini çalıştırdığınızdan emin olun.")
    print("\n💡 İpucu: 'Run All' yerine exercise hücrelerini tek tek çalıştırıp")
    print("   ardından bu hücreyi çalıştırmayı deneyin.")
    raise SystemExit()

# ══════════════════════════════════════════════════════════
# SEND SUBMISSION
# ══════════════════════════════════════════════════════════
_data = json.dumps({
    'week': WEEK,
    'studentId': _sid,
    'studentName': _sname,
    'studentEmail': _semail,
    'classCode': _scode,
    'source': 'cp2-notebook',
    'timeOnPage': _time_on_page,
    'wallTime': _timing['wall_time'],
    'cellsRun': _timing['total_cells_run'],
    'sessionCount': _timing['session_count'],
    'sessionStart': _timing['session_start'],
    'sessionEnd': _timing['session_end'],
    'timerSource': _timing['source'],
    'answers': _answers
}).encode('utf-8')

print("📡 Gönderiliyor...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get('success'):
        print(f"\n✅ {_result['message']}")
        print('📧 Onay için e-postanızı kontrol edin.')
    else:
        print(f"\n❌ {_result.get('message', 'Gönderim başarısız')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  İstek gönderildi — onay için e-postanızı kontrol edin.")
    print(f"(E-posta gelmezse tekrar deneyin veya hocaya başvurun)")
